In [1]:
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score
import numpy as np
import pandas as pd
from pandas import Series
import os

In [2]:
def return_percentile(inputs, alpha):
    ordered = np.sort(inputs)
    lower = np.percentile(ordered, (100-alpha)/2)
    upper = np.percentile(ordered, alpha+((100-alpha)/2))
    mean = np.mean(ordered)
    return np.round(lower, 3), np.round(mean, 3), np.round(upper, 3)

In [18]:
doctor_result = pd.read_csv('/data/put_data/timliu/kidney/result_doctor_evaluation_revised_160.csv')
doctor_result.head()

,level_0,index,path,type,uid,chtno,name,contentdate,year,uid_date,...,age,img,length,img_uid,Ans,Kuo,bCMUH1,bCMUH2,bCMUH3,cnn
0,174,2044,1015_I00179245953.png,right,1015_I00179245953,1015,I00179245953,20141027,2014,1015_20141027,...,69,1015_I00179245953.png,8.47,001.png,1,1,1,1,1,1
1,124,1285,1063_I00223224469.png,left,1063_I00223224469,1063,I00223224469,20151204,2015,1063_20151204,...,66,1063_I00223224469.png,11.76,002.png,1,1,1,1,0,1
2,179,2139,1121_I00203576197.png,left,1121_I00203576197,1121,I00203576197,20150625,2015,1121_20150625,...,83,1121_I00203576197.png,12.85,003.png,1,1,1,0,0,1
3,453,5410,1144_I00171871652.png,right,1144_I00171871652,1144,I00171871652,20140811,2014,1144_20140811,...,76,1144_I00171871652.png,9.83,004.png,1,1,1,1,0,1
4,167,1988,1200_I00182987138.png,left,1200_I00182987138,1200,I00182987138,20141204,2014,1200_20141204,...,49,1200_I00182987138.png,10.19,005.png,1,1,1,1,1,1


## Process each user's prediction

In [22]:
for c in doctor_result.columns[-5:]:
    print('Now process {}'.format(c))
    precision_list = []
    recall_list = []
    fscore_list = []
    accuracy_list = []
    for i in range(10000):
        unique_test_id = list(set(doctor_result.uid_date))
        bootstrap_index = np.random.choice(len(unique_test_id), len(unique_test_id), replace=True)
        select_id = [unique_test_id[i] for i in bootstrap_index]
        sub_df = pd.DataFrame([row for index, row in doctor_result.iterrows() if row.uid_date in select_id])
        
        precision, recall, fscore, support = score(sub_df.Ans, sub_df[['{}'.format(c)]])
        accuracy = accuracy_score(sub_df.Ans, sub_df[['{}'.format(c)]])
        
        precision_list.append(precision[1])
        recall_list.append(recall[1])
        fscore_list.append(fscore[1])
        accuracy_list.append(accuracy)

    dd = {'precision': precision_list, 
         'recall': recall_list,
         'fscore': fscore_list,
         'accuracy': accuracy_list}
    
    ddd = pd.DataFrame(data=dd)
    ddd.to_csv('CI_result/{}.csv'.format(c), index=False)
    print(ddd.shape)
    print('Done!')

Now process Kuo
(10000, 4)
Done!
Now process bCMUH1
(10000, 4)
Done!
Now process bCMUH2
(10000, 4)
Done!
Now process bCMUH3
(10000, 4)
Done!
Now process cnn
(10000, 4)
Done!


## Compute average

In [23]:
kuo = pd.read_csv('CI_result/Kuo.csv')
d1 = pd.read_csv('CI_result/bCMUH1.csv')
d2 = pd.read_csv('CI_result/bCMUH2.csv')
d3 = pd.read_csv('CI_result/bCMUH3.csv')

precision_avg = np.mean(kuo.precision + d1.precision + d2.precision + d3.precision) 
recall_avg = np.mean(kuo.recall + d1.recall + d2.recall + d3.recall) 
fscore_avg = np.mean(kuo.fscore + d1.fscore + d2.fscore + d3.fscore) 
accuracy_avg = np.mean(kuo.accuracy + d1.accuracy + d2.accuracy + d3.accuracy) 

doctor_avg = {'precision': precision_avg, 
              'recall': recall_avg,
              'fscore': fscore_avg,
              'accuracy': accuracy_avg}

doctor_avg_df = pd.DataFrame(data=doctor_avg)
doctor_avg_df.to_csv('CI_result/doctor_avg.csv', index=False)

## 95 confidence interval

In [27]:
csv_path = []
for dirName, dirPath, fileList in os.walk('CI_result'):
    for file in fileList:
        if 'csv' in file and 'summary' not in file:
            csv_path.append(os.path.join(dirName, file))
# check
csv_path[0]

'CI_result/bCMUH2.csv'

In [28]:
precision_percent = []
recall_percent = []
fscore_percent = []
accuracy_percent = []
for csv in csv_path:
    print(csv)
    df = pd.read_csv(csv)
    lower_precision, mean_precision, upper_precision = return_percentile(df.precision, 95)
    lower_recall, mean_recall, upper_recall = return_percentile(df.recall, 95)
    lower_fscore, mean_fscore, upper_fscore = return_percentile(df.fscore, 95)
    lower_accuracy, mean_accuracy, upper_accuracy = return_percentile(df.accuracy, 95)
    precision_percent.append('{} ({}, {})'.format(mean_precision, lower_precision, upper_precision))
    recall_percent.append('{} ({}, {})'.format(mean_recall, lower_recall, upper_recall))
    fscore_percent.append('{} ({}, {})'.format(mean_fscore, lower_fscore, upper_fscore))
    accuracy_percent.append('{} ({}, {})'.format(mean_accuracy, lower_accuracy, upper_accuracy))    
    
summary = {'User': csv_path, 'Precision (95% CI)': precision_percent,
           'Recall (95% CI)': recall_percent, 'F1 score (95% CI)': fscore_percent,
           'Accuracy (95% CI)': accuracy_percent}
summary_df = pd.DataFrame(data=summary)
summary_df.User = summary_df.User.apply(lambda x: x.split('/')[-1].replace('.csv', ''))
summary_df

CI_result/bCMUH2.csv
CI_result/doctor_avg.csv
CI_result/bCMUH3.csv
CI_result/bCMUH1.csv
CI_result/Kuo.csv
CI_result/cnn.csv


,Accuracy (95% CI),F1 score (95% CI),Precision (95% CI),Recall (95% CI),User
0,"0.65 (0.593, 0.707)","0.735 (0.683, 0.785)","0.917 (0.875, 0.963)","0.614 (0.549, 0.679)",bCMUH2
1,"0.705 (0.678, 0.731)","0.782 (0.759, 0.806)","0.912 (0.891, 0.934)","0.701 (0.672, 0.73)",doctor_avg
2,"0.607 (0.549, 0.667)","0.68 (0.619, 0.737)","0.957 (0.927, 1.0)","0.528 (0.461, 0.595)",bCMUH3
3,"0.763 (0.713, 0.814)","0.844 (0.806, 0.881)","0.881 (0.838, 0.929)","0.811 (0.761, 0.864)",bCMUH1
4,"0.8 (0.753, 0.848)","0.871 (0.836, 0.904)","0.893 (0.851, 0.935)","0.85 (0.805, 0.899)",Kuo
5,"0.856 (0.816, 0.898)","0.909 (0.881, 0.937)","0.913 (0.877, 0.951)","0.906 (0.868, 0.947)",cnn


## Sorted

In [31]:
sort_list = ['Kuo', 'bCMUH1', 'bCMUH2', 'bCMUH3', 'doctor_avg', 'cnn']
summary_df.User = summary_df.User.astype('category')
summary_df.User.cat.set_categories(sort_list, inplace=True)
summary_df = summary_df.sort_values(['User'])

In [32]:
summary_df

,Accuracy (95% CI),F1 score (95% CI),Precision (95% CI),Recall (95% CI),User
4,"0.8 (0.753, 0.848)","0.871 (0.836, 0.904)","0.893 (0.851, 0.935)","0.85 (0.805, 0.899)",Kuo
3,"0.763 (0.713, 0.814)","0.844 (0.806, 0.881)","0.881 (0.838, 0.929)","0.811 (0.761, 0.864)",bCMUH1
0,"0.65 (0.593, 0.707)","0.735 (0.683, 0.785)","0.917 (0.875, 0.963)","0.614 (0.549, 0.679)",bCMUH2
2,"0.607 (0.549, 0.667)","0.68 (0.619, 0.737)","0.957 (0.927, 1.0)","0.528 (0.461, 0.595)",bCMUH3
1,"0.705 (0.678, 0.731)","0.782 (0.759, 0.806)","0.912 (0.891, 0.934)","0.701 (0.672, 0.73)",doctor_avg
5,"0.856 (0.816, 0.898)","0.909 (0.881, 0.937)","0.913 (0.877, 0.951)","0.906 (0.868, 0.947)",cnn


## Save to csv

In [29]:
summary_df.to_csv('CI_result/summary.csv', index=False)